# Lispy

Veja o tutorial do Peter Norvig em http://norvig.com/lispy.html. Vamos implementar
um interpretador de Lisp/Scheme em Python.

In [10]:
!pip install lark-parser

     |████████████████████████████████| 286kB 4.8MB/s 
  Stored in directory: /root/.cache/pip/wheels/62/4a/21/4a4ccbef66bdfe3f71a1624babacaa869d6ec0c2f2bc1ee37e
Successfully built lark-parser


In [0]:
# Imports

import math
import operator as op
from collections import deque, ChainMap
from lark import Lark, InlineTransformer

# Apelidos
Symbol = str    # símbolo na linguagem
Ast = list      # árvore sintática
Ctx = ChainMap  # contexto de execução

In [0]:
class Symbol:
  __init__ = lambda self, s: setattr(self, 'name', s)
  __repr__ = lambda self: self.name
  __eq__ = lambda self, other: isinstance(other, Symbol) and self.name == other.name
  __hash__ = lambda self: -hash(self.name)

In [0]:
#
# Lark
#
class LispyTransformer(InlineTransformer):
    int = int
    float = float
    string = lambda self, s: s[1:-1]
    bool = lambda self, s: s=='#t'
    symbol = Symbol
    
    list = lambda self, *args: args 
    
    brack = lambda self, a, b, *args: (b, a, *args) 
    
    


grammar = Lark(r"""
?start : exp

?exp: list | atom | brack

list: "(" exp* ")" 

brack: "[" exp exp+ "]"

?atom : INT -> int
      | FLOAT -> float
      | STRING -> string
      | BOOL -> bool
      | SYMBOL -> symbol

INT : /\d+/
FLOAT: /\d+\.\d+/
STRING: /"[^\n"]*"/
BOOL: /#[tf]/
SYMBOL: /(?!\d)[-\w+_\/*%=<>!&?]+/
COMMENT: /;[^\n]*/


SPACE: /\s+/
%ignore SPACE
%ignore COMMENT

""", parser='lalr', transformer=LispyTransformer())



In [120]:
#
# Exemplos
#
ex = '''
(define fat (lambda (n) 
    (if [n < 2] 
        1
        [n * (fat [n - 1])])))
'''

#ex = '(#t)'

tree = parse(ex)
print(tree)

fat = eval_scheme(ex)
print(fat)
fat(5)

(define, fat, (lambda, (n,), (if, (<, n, 2), 1, (*, n, (fat, (-, n, 1))))))
<function run_ast.<locals>.function at 0x7f89fb76c048>


120

In [113]:
Symbol('if') == Symbol('if')
d = {
    Symbol('a'): 'b'
}
d[Symbol('a')]

'b'

# Análise sintática + léxica

A principal função é eval_scheme(), que analisa uma string de código e retorna a árvore sintática correspondente.

In [0]:
def eval_scheme(st: str, ctx=None):
    """
    Avalia a string de código Scheme no contexto padrão.
    """
    ctx = default_context if ctx is None else ctx
    ast = parse(st)
    return run_ast(ast, ChainMap({}, ctx))


def parse(st: str) -> Ast:
    """
    Realiza análise sintática da string de código e retorna uma 
    árvore sintática
    """
    return grammar.parse(st)
    
    
    
# Contexto de execução
default_context = {
    Symbol('+'): op.add,
    Symbol('-'): op.sub,
    Symbol('*'): op.mul,
    Symbol('/'): op.truediv,
    Symbol('<'): op.lt,
    Symbol('<='): op.le,
    Symbol('>'): op.gt,
    Symbol('>='): op.ge,
    Symbol('sqrt'): math.sqrt,
    Symbol('='): op.eq,
}

# Interpretador

O interpretador consiste em uma única função que recebe uma árvore sintática e executa os comandos correspondentes no contexto dado.

In [0]:
def run_ast(ast: Ast, ctx: Ctx):
    """
    Executa árvore sintática no contexto de execução dada.
    """
    
    if isinstance(ast, (float, int, str, bool)):
        return ast
    elif isinstance(ast, Symbol):
        try:
            return ctx[ast]
        except KeyError:
            raise NameError(f'unknown variable: {ast}')
    
    head, *args = ast
    if head == Symbol('define'):
        name, value = args
        ctx[name] = result = run_ast(value, ctx)
        return result

    elif head == Symbol('if'):
        cond, true, false = args
        cond = run_ast(cond, ctx)
        if cond:
            return run_ast(true, ctx)
        else:
            return run_ast(false, ctx)
        
    elif head == Symbol('lambda'):
        arg_names, body = args
        
        def function(*arg_values):
            local = dict(zip(arg_names, arg_values))
            new_ctx = ChainMap(local, ctx)
            return run_ast(body, new_ctx)
        return function
    
    else:
        func = run_ast(head, ctx)
        args = [run_ast(arg, ctx) for arg in args]
        return func(*args)